In [1]:
import ROOT
import numpy as np
import toml
from tqdm.notebook import tqdm, trange
from utils import *
import numba

%jsroot on

Welcome to JupyROOT 6.28/00


## Load Config
The config holds all the information on how to construct the histograms and other general settings

In [2]:
config = toml.load("stage0_config.toml")

## Read Calibration
Read the detector calibration from csv file

In [3]:
file = open("calib/calib_apr.csv", "r")
file.readline()

numch = config['general']['numch']

calib_slope = [1] * numch
calib_offset = [0] * numch

for line in file:
    values = line.strip().split(',')
    calib_slope[int(values[0])] = float(values[1])
    calib_offset[int(values[0])] = float(values[2])

file.close()

calib_slope = np.array(calib_slope)
calib_offset = np.array(calib_offset)



## NUMBA Function
Create JIT compiled C++ function for creating hEgam column

In [4]:
@ROOT.Numba.Declare(["int","int"], "double")
def calc_Egam(detector,PulseHeight):
    return PulseHeight*calib_slope[detector]+calib_offset[detector]

## Create RDataFrame
Create data frame and define new columns then filter by channel. Create histogram models and then fill histograms.

In [5]:
ROOT.EnableImplicitMT()

tree_name = config['general']['tree_name']
numch = config['general']['numch']
fp_length = config['general']['fp_length']
pu_flag = config['general']['pu_flag']
fo_flag = config['general']['fo_flag']

# file_name = "data/rawroot_run_0064*"
file_name = "data/rawroot_run_0014*"


df = ROOT.RDataFrame(tree_name, file_name)

df = (df.Define("tof_ns","tof*10.0")
      .Define("tof_mus","tof_ns/1000.0+0.0000001")
      .Define("En","pow((72.3*21.5/(tof_ns/1000.0)),2)+0.00001")
      .Define("Egam","Numba::calc_Egam(detector,PulseHeight)"))

df_ch = []

for ch in range(numch):
    
    df_ch.append(df.Filter(f"detector == {ch}"))

hist_dict = {}

for key, hist_conf in config['hist'].items():
    
    # name = hist_conf['name']
    xaxis = hist_conf['xaxis']
    yaxis = hist_conf['yaxis']
    
    col = hist_conf['col']
    gate = hist_conf['gate']
    
    down = hist_conf['bins']['down']
    up = hist_conf['bins']['up']
    N = hist_conf['bins']['N']
    var = hist_conf['bins']['var']
    
    df_gate_ch = []
    if gate != "":
        df_gate = df.Filter(gate)
        for ch in range(numch):
            df_gate_ch.append(df_gate.Filter(f"detector == {ch}"))
    else:
        df_gate = df
        df_gate_ch = df_ch
        

    if hist_conf['all']:

        name = hist_conf['name']
        title  = name + ';' + xaxis + ';' + yaxis
        xbins = get_xbins(N, down, up, var, fp_length)

        hist_model_all = ROOT.RDF.TH1DModel(name, title, N, xbins)
        hist_all = df_gate.Histo1D(hist_model_all, col)

        hist_dict[key] = hist_all

    else:
        hist_dict[key] = {}
        for ch in range(numch):
            
            name = hist_conf['name'] + f'_d{ch}'
            title  = name + ';' + xaxis + ';' + yaxis
            xbins = get_xbins(N, down, up, var, fp_length)

            if 'hEgam' in name:
                scale = calib_slope[ch]
            else:
                scale = 1

            hist_model_ch = ROOT.RDF.TH1DModel(name, title, N, xbins*scale)
            hist_ch = df_gate_ch[ch].Histo1D(hist_model_ch, col)

            hist_dict[key][name] = hist_ch
        
        
# hist_dict


## Write Histograms
Write histograms from dictionary to a root file maintaing heirerachy. 

In [6]:
# read_root_to_dict("test.root")

In [7]:
det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,19,-1,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] # det number for index of position number runs 13~54
# det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,-1,19,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] #det number for index of position number runs 55~90

# filename = "stage0_output.root"

det_pos_map = ROOT.TH2I("det_pos_map","det_pos_map;position # ;detector # ",32,0,32,32,0,32)
det_pos_graph = ROOT.TGraph()
det_pos_graph.SetTitle("det_pos_graph")
det_pos_graph.SetName("det_pos_graph")
det_pos_graph.GetXaxis().SetTitle("positon #")
det_pos_graph.GetYaxis().SetTitle("detector #")



for ch in range(numch):
    det_pos_map.SetBinContent(ch+1,det_number[ch]+1,1)
    det_pos_graph.AddPoint(ch,det_number[ch])

# hist_dict[det_pos_map.GetName()] = det_pos_map
# hist_dict[det_pos_graph.GetName()] = det_pos_graph


# with ROOT.TFile(filename,"UPDATE") as outfile:

#     if det_pos_graph.GetName() in outfile.GetListOfKeys():
#         # print(outfile.GetListOfKeys())
#         existing_histogram = outfile.Get(det_pos_map.GetName())
#         existing_histogram.Delete()

#     if det_pos_map.GetName() in outfile.GetListOfKeys():
#         existing_histogram = outfile.Get(det_pos_map.GetName())
#         existing_histogram.Delete()

#     # outfile.WriteObject(det_pos_graph, det_pos_graph.GetName())
    
#     det_pos_graph.Write()
#     det_pos_map.Write()

In [8]:
write_dict_to_root(hist_dict, "stage0_output.root")

In [9]:
# read_root_to_dict("stage0_output.root")